In [1]:
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
# start_idx: 시작 페이지
# show_cnt: 보여줄 개수
params = {'start_idx':'1', 'show_cnt': '50'}
# 건강기능식품 검색 url
res = requests.get('https://www.foodsafetykorea.go.kr/portal/healthyfoodlife/searchHomeHFProc.do', params=params)

In [3]:
# circed digit -> digit 
# 0x2460 ~ 0x2473: ① ~ ⑳
def uncircle(s):
    for i in range(1, 21):
        s = s.replace(chr(0x245f + i), str(i)+". ")
    return s

In [4]:
data = []

if res.status_code == 200:
    for i in tqdm(range(50)):
        # response의 json에서 고유 번호 추출
        no = res.json()[i]['prdlst_report_ledg_no']
        # 상세 페이지 url
        res_detail = requests.get('https://www.foodsafetykorea.go.kr/portal/healthyfoodlife/searchHomeHFDetail.do?prdlstReportLedgNo=' + no)
        if res_detail.status_code == 200:
            soup = BeautifulSoup(res_detail.content, 'html.parser')

            row = {}

            for tbody in soup.find_all('tbody'):
                for tr in tbody.find_all('tr'):
                    th = tr.find('th')
                    td = tr.find('td')
                    if th:
                        row['url'] = 'https://www.foodsafetykorea.go.kr/portal/healthyfoodlife/searchHomeHFDetail.do?prdlstReportLedgNo=' + no
                        if th.text == '신고번호':
                            row['제품코드'] = 'kfood-' + td.text
                        elif th.text == '제품명':
                            row['제품명'] = td.text
                        elif th.text == '업소명':
                            row['제조사'] = td.text
                        elif th.text == '성상':
                            row['제형'] = td.text.split()[-1]
                        elif th.text == '섭취량/섭취 방법':
                            row['섭취방법'] = td.text
                        elif th.text == '섭취시주의사항':
                            row['주의사항'] = td.text
                        elif th.text == '기준 및 규격':
                            ingred = []
                            #'1. ', '(1) ', '1) ', '(1)'
                            for i in re.split('\d+\. |\(\d+\) |\d+\) |\(\d+\)', uncircle(td.text)):
                                #필터링
                                fltr = '성상|붕해시험|대장균군|붕해도|붕해규격|붕해실험|붕해'
                                if re.search(fltr, i):
                                    continue
                                #' : ', ': '
                                split_td = re.split('\s?\:\s', i)
                                if len(split_td) > 1:
                                    ingred.append([split_td[0], split_td[1]])
                            # 리스트 원소만 가져오기
                            row['성분'] = str(ingred)[1:-1] if len(ingred) > 0 else ''  

            data.append(row)


100%|███████████████████████████████████████████| 50/50 [00:47<00:00,  1.05it/s]


In [5]:
columns=['제품코드', '제품명', '제조사', '제형', '섭취방법', '주의사항', 'url', '성분']

df = pd.DataFrame(data=data, columns=columns)
df.to_excel('result.xlsx', index=False)